In [1]:
#CNN Refined Code
## Importing whatever required
import tensorflow as tf 
import numpy as np
import pandas as pd
sess = tf.InteractiveSession()
from tempfile import TemporaryFile
outfile = TemporaryFile()
import pickle

In [2]:
#method to load GloVe Vectors by using the now keyword 'model'
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

In [3]:
model = loadGloveModel('glove.6B.50d.txt') # loading word embeddings

Loading Glove Model
Done. 400001  words loaded!


In [4]:
questions_as_words = np.array(pd.read_csv('./myra/data4.csv')['0']) #loading the questions

In [5]:
#converting questions to word vectors
questions_as_vecs = []
for line in questions_as_words:
    sentence_vecs = np.zeros((50,20))
    line = line.lower()
    line_split = line.split()
    i = 0;
    for word in line_split: #obtaining embeddings for all the words in a line and putting them in a vector 50*20
                            # limiting the number of words in a sentence to 20
        try:
            sentence_vecs[:,i] = model[word]
            i+=1    
        except:
            sentence_vecs[:,i] = model['<unk>']
            i+=1 
        if i == 19:
            break;
    questions_as_vecs.append(sentence_vecs) #putting the individual vectors in a larger vector
vectors = np.reshape(questions_as_vecs,(1409,50,20,1))    # converting question vectors to tensor format
pickle.dump(vectors, open("./myra/question_vectors.p", "wb")) # saving the sentence embeddings        


In [6]:
vecs = pickle.load( open( "./myra/question_vectors.p", "rb" ) )
vectors = np.reshape(vecs,(1409,50,20,1))

In [7]:
question_labels = np.array(pd.read_csv('./myra/data4.csv')['1']) # reading the sentence labels
size = len(question_labels)
labels_one_hot = np.zeros((size,282)) # a 282 dimensional label vector for the dataset
labels_one_hot[np.arange(size),question_labels] = 1 # converting labels to one hot representation

In [8]:
#Placeholders for training data
x = tf.placeholder(tf.float32, shape=[None,50,20,1])
y_ = tf.placeholder(tf.float32, shape=[None,282])
# weights
w_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1), tf.float32)
b_conv1 = tf.Variable(tf.constant(0.1, tf.float32, shape=[32]))

convolve1 = tf.nn.conv2d(x,w_conv1,strides=[1,1,1,1], padding="SAME") + b_conv1
h_conv1 = tf.nn.relu(convolve1)
max_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

layer1 = max_pool1
print "layer1 : ",layer1


<tf.Tensor 'MaxPool:0' shape=(?, 25, 10, 32) dtype=float32>

In [9]:
w_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1), tf.float32)
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]), tf.float32)

convolve2 = tf.nn.conv2d(layer1,w_conv2, strides=[1,1,1,1], padding="SAME") + b_conv2
relu2 = tf.nn.relu(convolve2)
max_pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
layer2 = max_pool2

layer2_mat = tf.reshape(layer2,[-1,13*5*64])

In [10]:
w_fc1 = tf.Variable(tf.truncated_normal([13*5*64,1024], stddev=0.1), tf.float32)
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]), tf.float32)

fully_1 = tf.matmul(layer2_mat, w_fc1) + b_fc1
relu3 = tf.nn.relu(fully_1)
layer3 = relu3
print layer3

Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)


In [11]:
w_fc2 = tf.Variable(tf.truncated_normal([1024,282], stddev=0.1), tf.float32)
b_fc2 = tf.Variable(tf.constant(0.1, shape=[282]), tf.float32)
#layer3_drop = tf.nn.dropout(layer3,0.2)
fully_2 = tf.matmul(layer3, w_fc2) + b_fc2
softy = tf.nn.softmax(fully_2)
layer4 = softy
print layer4

Tensor("Softmax:0", shape=(?, 282), dtype=float32)


In [12]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(layer4), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(layer4,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [13]:
def batches(iterable, n = 1):
   current_batch = []
   for item in iterable:
       current_batch.append(item)
       if len(current_batch) == n:
           yield current_batch
           current_batch = []
   if current_batch:
       yield current_batch

In [ ]:
for i in range(200):
  # for batch,batch_label in zip(batches(vectors, 1),batches(labels_one_hot,1)):
   train_step.run(feed_dict={x:batch, y_:batch_label}) 
   if i%10 == 0:
       train_accuracy = accuracy.eval(feed_dict={x:vectors, y_:labels_one_hot})
       print "step %d: train_accuracy = %g"%(i, float(train_accuracy))
     